# **Utiliser les Modèles de Langage N-gram pour générer un texte**

**Par :** *Nouha EL MHAMDI*

**Objectif de l'atelier :**
Cet atelier vise à comprendre le principe de la génération de texte à l’aide d’un modèle de langage basé sur les N-grammes. L'objectif est de comprendre comment ce modèle apprend les probabilités des séquences de mots, comment il est entraîné sur un corpus textuel, et comment le choix du paramètre $n$ influence la richesse et la cohérence du texte généré.

## **1. Préparation et Corpus**

### **Étape 1 : Importation des bibliothèques**

Nous importons les bibliothèques nécessaires pour le traitement du langage naturel, les structures de données et les opérations mathématiques.



In [3]:
import re
import math
import random
from collections import Counter, defaultdict



### **Corpus étudié**

Le corpus fourni pour l'entraînement du modèle est défini ci-dessous.



In [4]:
Corpus = [
    "L'apprentissage automatique est un sous-domaine de l'intelligence artificielle qui permet aux ordinateurs d'apprendre à partir de données.",
    "Le Maroc est un pays situé en Afrique du Nord. Sa capitale est Rabat. Il possède une riche histoire.",
    "Rabat est la capitale du Maroc et est connue pour sa culture, son histoire et sa gastronomie.",
    "L'informatique est la science du traitement automatique de l'information par des machines."
]



## 2. **Nettoyage et Tokenisation**

### **Étape 2 : Définition de la fonction de tokenisation**

Cette fonction prend une chaîne de caractères, la convertit en minuscules et la divise en tokens (mots) en gérant les caractères spéciaux et les apostrophes.



In [5]:
def tokenize(text):
    # Convertir en minuscules
    text = text.lower()
    # Remplacer les caractères non alphabétiques (sauf l'apostrophe) par un espace
    text = re.sub(r"[^a-zàâçéèêëiîïôùûüÿñ\s']", ' ', text) 
    # Remplacer les apostrophes par un espace pour séparer les mots attachés (ex: l'apprentissage -> l apprentissage)
    text = re.sub(r"'", ' ', text) 
    
    tokens = [t for t in text.split() if t.strip()]
    return tokens



### **Séparation du corpus en phrases**

Nous séparons le corpus en phrases en utilisant le point (`.`) comme délimiteur, ce qui est une étape cruciale pour entraîner un modèle de langage sur des séquences de mots au sein d'une même unité sémantique.



In [6]:
sentences = []
for doc in Corpus:
    for s in doc.split('.'):
        s = s.strip()
        if len(s) > 0:
            sentences.append(s)



### **Test de l'étape 2**

Exécution de la séparation en phrases et test de la fonction de tokenisation.



In [7]:
print(f"Nombre total de phrases: {len(sentences)}")
print("Liste des phrases:")
print(sentences)
print("\nTest de tokenisation sur la première phrase:")
print(tokenize(sentences[0]))

Nombre total de phrases: 6
Liste des phrases:
["L'apprentissage automatique est un sous-domaine de l'intelligence artificielle qui permet aux ordinateurs d'apprendre à partir de données", 'Le Maroc est un pays situé en Afrique du Nord', 'Sa capitale est Rabat', 'Il possède une riche histoire', 'Rabat est la capitale du Maroc et est connue pour sa culture, son histoire et sa gastronomie', "L'informatique est la science du traitement automatique de l'information par des machines"]

Test de tokenisation sur la première phrase:
['l', 'apprentissage', 'automatique', 'est', 'un', 'sous', 'domaine', 'de', 'l', 'intelligence', 'artificielle', 'qui', 'permet', 'aux', 'ordinateurs', 'd', 'apprendre', 'à', 'partir', 'de', 'données']




**Analyse et Interprétation**


1.  **Séparation en phrases :** Le résultat indique 6 phrases au total. Cela signifie que le corpus initial, bien que court, a été correctement segmenté en six unités d'entraînement distinctes en utilisant le point comme délimiteur. Chaque phrase sera traitée comme une séquence indépendante pour l'entraînement du modèle.

2.  **Nettoyage (`tokenize`) :** Le test sur la première phrase montre que la fonction tokenize a bien séparé les mots. Par exemple, l'expression L'apprentissage est divisée en deux tokens : l et apprentissage. Cette approche simplifiée est courante dans les modèles N-gram pour s'assurer que chaque composant du mot (comme l'article élidé) est compté séparément.
## **3. Comptage des N-grammes**

### **Étape 3 : Définition des variables globales et de la fonction de comptage**

Nous définissons l'ordre du modèle $n$ (initialement $n=2$ pour le Bigramme) et les compteurs pour les N-grammes et les contextes.



In [8]:
# Définir n (ordre du modèle N-gram)
n = 2 

# Initialisation des compteurs
ngram_counts = Counter()
context_counts = Counter()
vocab = set()
V = 0 # Taille du vocabulaire, sera calculée après le comptage

def count_ngrams(n_order):
    """
    Calcule les N-grammes et les contextes pour un ordre n donné.
    Met à jour les variables globales ngram_counts, context_counts, vocab et V.
    """
    global n, ngram_counts, context_counts, vocab, V
    
    n = n_order
    ngram_counts.clear()
    context_counts.clear()
    vocab.clear()
    
    for s in sentences:
        tokens = tokenize(s)
        
        # Mise à jour du vocabulaire
        vocab.update(tokens)
        
        # Ajout des tokens de début de phrase <s> et de fin de phrase </s>
        start_tokens = ["<s>"] * (n - 1)
        tokens = start_tokens + tokens + ["</s>"]
        
        # Comptage des N-grammes et des contextes
        for i in range(len(tokens) - n + 1):
            ngram = tuple(tokens[i : i + n])
            context = tuple(tokens[i : i + n - 1])
            
            ngram_counts[ngram] += 1
            context_counts[context] += 1

    # Taille du vocabulaire (V) = nombre de mots uniques + 1 pour le token de fin de phrase </s>
    V = len(vocab) + 1 



### **Test de l'étape 3**

Exécution du comptage pour $n=2$ et affichage des résultats.



In [9]:
# Exécuter le comptage pour n=2
count_ngrams(2)

print(f"Ordre du modèle N-gram (n): {n}")
print(f"Taille du vocabulaire (V): {V}")
print(f"\nVocabulaire (sans <s> et </s>): {sorted(list(vocab))}")
print(f"\nNombre total de N-grammes (Top 5):")
print(ngram_counts.most_common(5))
print(f"\nNombre total de contextes (Top 5):")
print(context_counts.most_common(5))

Ordre du modèle N-gram (n): 2
Taille du vocabulaire (V): 50

Vocabulaire (sans <s> et </s>): ['afrique', 'apprendre', 'apprentissage', 'artificielle', 'automatique', 'aux', 'capitale', 'connue', 'culture', 'd', 'de', 'des', 'domaine', 'données', 'du', 'en', 'est', 'et', 'gastronomie', 'histoire', 'il', 'information', 'informatique', 'intelligence', 'l', 'la', 'le', 'machines', 'maroc', 'nord', 'ordinateurs', 'par', 'partir', 'pays', 'permet', 'possède', 'pour', 'qui', 'rabat', 'riche', 'sa', 'science', 'situé', 'son', 'sous', 'traitement', 'un', 'une', 'à']

Nombre total de N-grammes (Top 5):
[(('<s>', 'l'), 2), (('est', 'un'), 2), (('de', 'l'), 2), (('est', 'la'), 2), (('l', 'apprentissage'), 1)]

Nombre total de contextes (Top 5):
[(('<s>',), 6), (('est',), 6), (('l',), 4), (('de',), 3), (('du',), 3)]




**Analyse et Interprétation :**

1.  **Préparation :** Le code introduit les marqueurs de début de phrase (`<s>`) et de fin de phrase (`</s>`). Pour un bigramme ($n=2$), chaque phrase commence par un seul `<s>`. Ce modèle fait l'hypothèse que la probabilité d'un mot dépend uniquement du mot qui le précède immédiatement ($P(w_i | w_{i-1})$).


2.  **Comptage :** `ngram_counts` stocke la fréquence de chaque séquence de $n$ mots. `context_counts` stocke la fréquence de chaque séquence de $n-1$ mots (le contexte). càd Les Bigrammes les plus fréquents, comme (`<s>`, 'l') (2 occurrences), indiquent les séquences les plus courantes. Le Bigramme (`<s>`, 'l') signifie que deux phrases commencent par le mot l'(par exemple, "L'apprentissage..." et "L'informatique...").

3.  **Vocabulaire ($V$) :** La taille du vocabulaire est de 50. Cette valeur est cruciale. Elle est calculée en prenant les 49 mots uniques du corpus et en ajoutant le token de fin de phrase </s>. Ce $V=50$ sera utilisé dans la formule du lissage de Laplace.

## **4. Probabilité Conditionnelle avec Lissage de Laplace**

### **Étape 4 : Définition de la fonction de probabilité conditionnelle**

Nous définissons la fonction `ngram_prob(context, word)` qui retourne $P(\text{word} | \text{context})$ en utilisant le lissage de Laplace avec $\alpha=1$.

**Rôle du Lissage de Laplace :**

Le lissage de Laplace (ou *add-one smoothing*) est une technique de régularisation utilisée dans les modèles de langage N-gram pour **éviter le problème de la fréquence zéro**. Il garantit que même les N-grammes jamais vus se voient attribuer une petite probabilité non nulle, empêchant ainsi la probabilité d'une phrase entière de devenir nulle.

La formule utilisée est :
$$P(w_i | w_{i-n+1}^{i-1}) = \frac{C(w_{i-n+1}^{i}) + \alpha}{C(w_{i-n+1}^{i-1}) + \alpha \cdot V}$$

In [12]:
alpha = 1.0 # Paramètre de lissage de Laplace

def ngram_prob(context, word):
    """
    Calcule la probabilité conditionnelle P(word | context) avec lissage de Laplace.
    """
    context = tuple(context)
    ngram = context + (word,)
    
    # Numérateur: C(N-gramme) + alpha
    num = ngram_counts[ngram] + alpha
    
    # Dénominateur: C(Contexte) + alpha * V
    den = context_counts[context] + alpha * V
    
    return num / den



### **Test de l'étape 4**

Exécution de la fonction de probabilité pour différents cas.



In [13]:
# Assurez-vous que le comptage pour n=2 a été exécuté
count_ngrams(2)

# Exemple 1: Probabilité d'un N-gramme vu dans le corpus
print(f"Probabilité de 'maroc' sachant le contexte ('le',): {ngram_prob(('le',), 'maroc'):.4f}")

# Exemple 2: Probabilité d'un N-gramme vu dans le corpus
print(f"Probabilité de 'est' sachant le contexte ('le',): {ngram_prob(('le',), 'est'):.4f}")

# Exemple 3: Probabilité d'un mot inconnu ('pomme') - illustre le lissage
print(f"Probabilité de 'pomme' (mot inconnu) sachant le contexte ('le',): {ngram_prob(('le',), 'pomme'):.4f}")

Probabilité de 'maroc' sachant le contexte ('le',): 0.0392
Probabilité de 'est' sachant le contexte ('le',): 0.0196
Probabilité de 'pomme' (mot inconnu) sachant le contexte ('le',): 0.0196




**Analyse et Interprétation (Étape 4) :**

L'analyse des probabilités démontre l'efficacité du lissage de Laplace ($\alpha=1$).

- **Probabilité de 'maroc' sachant 'le' :** Le résultat est **0.0392**. Ce Bigramme `('le', 'maroc')` apparaît 1 fois dans le corpus. La probabilité est calculée en utilisant la formule du lissage : $\frac{C(\text{'le', 'maroc'}) + 1}{C(\text{'le'}) + V} = \frac{1 + 1}{1 + 50} = \frac{2}{51}$.

- **Probabilité de 'est' sachant 'le' :** Le résultat est **0.0196**. Ce Bigramme `('le', 'est')` n'apparaît **jamais** dans le corpus ($C=0$). La probabilité est donc $\frac{0 + 1}{1 + 50} = \frac{1}{51}$.

- **Probabilité de 'pomme' (mot inconnu) sachant 'le' :** Le résultat est également **0.0196**. Le mot `pomme` n'est pas dans le vocabulaire, et le Bigramme est donc inconnu.

**Conclusion sur le Lissage :** Le lissage de Laplace garantit que même les séquences jamais vues (comme `'le', 'est'`) ou les mots inconnus (`'pomme'`) reçoivent une probabilité non nulle de $\approx 0.0196$. Sans ce lissage, ces probabilités seraient de zéro, ce qui rendrait la probabilité de toute phrase contenant ces séquences nulle, un problème majeur dans les modèles N-gram.


## **5. Probabilités de Transition et Vecteur Initial**

### **Étape 5 : Trouver les probabilités de transition**

Les probabilités de transition sont les probabilités conditionnelles $P(w_i | w_{i-n+1}^{i-1})$ pour tous les mots $w_i$ du vocabulaire. Nous définissons une fonction pour lister les mots suivants les plus probables pour un contexte donné.



In [14]:
def predict_next(context, topk=5):
    """
    Prédit les top-k mots suivants pour un contexte donné en utilisant ngram_prob.
    """
    if isinstance(context, str):
        context = tokenize(context)
        
    # Tronquer le contexte pour qu'il corresponde à l'ordre n-1
    start_tokens = ["<s>"] * (n - 1)
    context = (start_tokens + context)[-n + 1:] 
    
    candidates = []
    # Calculer la probabilité pour chaque mot du vocabulaire (y compris </s>)
    full_vocab = vocab.union({"</s>"})
    
    for w in full_vocab:
        p = ngram_prob(context, w)
        candidates.append((w, p))
        
    # Trier par probabilité décroissante
    candidates.sort(key=lambda x: x[1], reverse=True)
    
    return candidates[:topk]



### **Test de l'étape 5**

Affichage des probabilités de transition pour deux contextes différents.



In [15]:
# Assurez-vous que le comptage pour n=2 a été exécuté
count_ngrams(2)

print(f"\nProbabilités de transition (Top 5) pour le contexte de début de phrase ('<s>',):")
print(predict_next(["<s>"]))

print(f"\nProbabilités de transition (Top 5) pour le contexte ('le',):")
print(predict_next(["le"]))


Probabilités de transition (Top 5) pour le contexte de début de phrase ('<s>',):
[('l', 0.05357142857142857), ('sa', 0.03571428571428571), ('il', 0.03571428571428571), ('le', 0.03571428571428571), ('rabat', 0.03571428571428571)]

Probabilités de transition (Top 5) pour le contexte ('le',):
[('maroc', 0.0392156862745098), ('connue', 0.0196078431372549), ('information', 0.0196078431372549), ('l', 0.0196078431372549), ('du', 0.0196078431372549)]


- **Probabilités de transition pour le contexte ('le',) :** Le mot `maroc` est le plus probable (0.0392) pour suivre `le`, car le Bigramme `('le', 'maroc')` est le seul Bigramme observé dans le corpus qui commence par `le`. Tous les autres mots listés (`connue`, `information`, `l`, `du`) ont la probabilité de base du lissage (0.0196), car ils n'ont jamais été vus directement après `le` dans le corpus.



### **Étape 6 : Trouver le vecteur des probabilités initiales**

Le vecteur des probabilités initiales est la distribution de probabilité des mots qui commencent une phrase. Il est obtenu en calculant les probabilités de transition pour le contexte de début de phrase.



In [16]:
# Le vecteur des probabilités initiales est P(w | <s>)
# Nous affichons les 5 plus probables
initial_probabilities = predict_next(["<s>"], topk=5) 

print(f"\nVecteur des probabilités initiales (Top 5):")
for word, prob in initial_probabilities:
    print(f"P('{word}' | '<s>') = {prob:.4f}")


Vecteur des probabilités initiales (Top 5):
P('l' | '<s>') = 0.0536
P('sa' | '<s>') = 0.0357
P('il' | '<s>') = 0.0357
P('le' | '<s>') = 0.0357
P('rabat' | '<s>') = 0.0357



**Analyse et Interprétation :**

Ces étapes préparent la phase de génération en identifiant les probabilités de passage d'un état (contexte) à un autre (mot suivant).

- **Vecteur des probabilités initiales :** Les mots les plus probables pour commencer une phrase sont `l` (0.0536), `sa`, `il`, `le`, et `rabat` (tous à 0.0357). La probabilité de `l` est plus élevée car il commence deux des six phrases du corpus, tandis que les autres mots n'en commencent qu'une seule. Ce vecteur est utilisé pour choisir le tout premier mot lors de la génération de texte.


## **6. Génération de Texte et Impact de $n$**

### **Étape 7 : Définition de la fonction de génération de texte**

Cette fonction implémente le processus de génération stochastique en utilisant les probabilités conditionnelles calculées. Elle permet de re-entraîner le modèle pour différents ordres $n$.



In [17]:
def generate_text(model_n, max_length=20):
    """
    Génère un texte en utilisant le modèle N-gram entraîné pour l'ordre model_n.
    """
    # Re-entraîner le modèle pour l'ordre n spécifié
    local_n = model_n
    local_ngram_counts = Counter()
    local_context_counts = Counter()
    local_vocab = set()

    for s in sentences:
        tokens = tokenize(s)
        local_vocab.update(tokens)
        start_tokens = ["<s>"] * (local_n - 1)
        tokens = start_tokens + tokens + ["</s>"]
        
        for i in range(len(tokens) - local_n + 1):
            ngram = tuple(tokens[i : i + local_n])
            context = tuple(tokens[i : i + local_n - 1])
            local_ngram_counts[ngram] += 1
            local_context_counts[context] += 1
            
    local_V = len(local_vocab) + 1 
    local_full_vocab = local_vocab.union({"</s>"})
    
    # Fonction de probabilité locale (pour éviter les dépendances aux globales)
    def local_ngram_prob(context, word):
        context = tuple(context)
        ngram = context + (word,)
        num = local_ngram_counts[ngram] + alpha
        den = local_context_counts[context] + alpha * local_V
        return num / den

    # Fonction de prédiction locale
    def local_predict_next(context):
        if isinstance(context, str):
            context = tokenize(context)
            
        start_tokens = ["<s>"] * (local_n - 1)
        context = (start_tokens + context)[-local_n + 1:] 
        
        candidates = []
        for w in local_full_vocab:
            p = local_ngram_prob(context, w)
            candidates.append((w, p))
            
        candidates.sort(key=lambda x: x[1], reverse=True)
        return candidates

    # Initialisation de la génération
    generated_tokens = []
    current_context = tuple(["<s>"] * (local_n - 1))
    
    for _ in range(max_length):
        candidates = local_predict_next(list(current_context))
        
        words = [w for w, p in candidates]
        probabilities = [p for w, p in candidates]
        
        total_prob = sum(probabilities)
        if total_prob == 0:
            break 
        probabilities = [p / total_prob for p in probabilities]
        
        # Choix stochastique du mot suivant
        next_word = random.choices(words, weights=probabilities, k=1)[0]
        
        if next_word == "</s>":
            break
            
        generated_tokens.append(next_word)
        # Mise à jour du contexte: on garde les (n-1) derniers mots
        current_context = current_context[1:] + (next_word,)
        
    return " ".join(generated_tokens).capitalize()



### **Test de l'étape 7 : Observation de l'effet de $n$**

Nous testons la génération de texte pour $n=1$ (Unigramme), $n=2$ (Bigramme) et $n=3$ (Trigramme).



In [18]:
# Fixer la graine pour la reproductibilité des tests
random.seed(42) 

print("\n--- Génération de texte (n=1) ---")
print(f"Modèle Unigramme (n=1): {generate_text(1)}")

print("\n--- Génération de texte (n=2) ---")
print(f"Modèle Bigramme (n=2): {generate_text(2)}")

print("\n--- Génération de texte (n=3) ---")
print(f"Modèle Trigramme (n=3): {generate_text(3)}")


--- Génération de texte (n=1) ---
Modèle Unigramme (n=1): D information informatique de la domaine automatique histoire rabat information sa apprentissage information machines traitement est de un et connue

--- Génération de texte (n=2) ---
Modèle Bigramme (n=2): Situé domaine qui riche aux qui l apprentissage intelligence une et données maroc en pour est et à intelligence est

--- Génération de texte (n=3) ---
Modèle Trigramme (n=3): Domaine l de pays du de riche informatique science d d sa informatique nord traitement à apprendre la apprendre d




**Analyse et Interprétation :**

Les résultats de la génération de texte illustrent le compromis fondamental dans le choix de l'ordre $n$.

- **Modèle Unigramme ($n=1$) :** Le texte généré (`D information informatique de la domaine...`) est une suite de mots sans aucune cohérence grammaticale ou sémantique. Cela est normal, car le modèle Unigramme choisit chaque mot indépendamment de son contexte, se basant uniquement sur la fréquence d'apparition du mot dans le corpus.

- **Modèle Bigramme ($n=2$) :** Le texte (`Situé domaine qui riche aux qui l apprentissage...`) montre une **cohérence locale** (les paires de mots sont souvent correctes), mais la phrase reste globalement incompréhensible. Le Bigramme capture les dépendances entre deux mots consécutifs, mais échoue à maintenir une structure de phrase sur le long terme.

- **Modèle Trigramme ($n=3$) :** Le texte (`Domaine l de pays du de riche informatique science d d sa informatique nord...`) est plus **cohérent** localement, mais on observe un phénomène de **surapprentissage**. Avec un corpus aussi petit, le modèle Trigramme a tendance à reproduire des fragments exacts du texte d'entraînement (`informatique science d`), car il n'a pas assez de données pour généraliser.

**Conclusion Pédagogique sur l'Impact de $n$ :**

L'ordre $n$ est un hyperparamètre critique.

- Un **petit $n$** (comme $n=1$) offre une grande **généralisation** (il peut créer des séquences jamais vues) mais une très faible **cohérence**.
- Un **grand $n$** (comme $n=3$) offre une meilleure **cohérence locale** mais souffre de la **rareté des données** et du **surapprentissage**, limitant sa capacité à générer un texte vraiment nouveau.

Le choix optimal de $n$ est toujours un compromis entre ces deux aspects. Pour un corpus plus grand, un $n$ plus élevé pourrait être justifié.

**Limites du Modèle N-gram :**

*   **Problème de la rareté des données :** La croissance exponentielle du nombre de N-grammes avec $n$ rend le modèle impraticable pour de grandes valeurs de $n$.
*   **Dépendance limitée :** Le modèle ne peut capturer que les dépendances locales (fenêtre de $n$ mots). Il ne peut pas modéliser les dépendances à longue distance (par exemple, le sujet et le verbe séparés par plusieurs mots).
*   **Modèle de Markov :** Il suppose que le mot suivant ne dépend que des $n-1$ mots précédents, ignorant tout le contexte antérieur.

Ces limites ont motivé le développement des modèles de langage neuronaux (comme les Transformers) qui sont devenus la norme actuelle en PNL.
